In [1]:
from pathlib import Path as Path
from torch.utils import data as data
from PIL import Image as PILImage
import torch

Creating a class of PascalVOC

In [2]:
from torchvision.datasets import VOCDetection

In [ ]:
data = VOCDetection(root='../data/PascalVOC',download=True)

0.0%

0.2%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

0.5%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

1.1%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

1.5%IOPub message rate exceeded.
T

In [32]:
sorted((Path.cwd()/'annotations').glob('*'))

[PosixPath('/Users/alexxcollins/OneDrive - City, University of London/modules/computer_vision/labs/OneDrive_1_22-02-2022/annotations/.DS_Store'),
 PosixPath('/Users/alexxcollins/OneDrive - City, University of London/modules/computer_vision/labs/OneDrive_1_22-02-2022/annotations/2007_000032.xml'),
 PosixPath('/Users/alexxcollins/OneDrive - City, University of London/modules/computer_vision/labs/OneDrive_1_22-02-2022/annotations/2007_000033.xml'),
 PosixPath('/Users/alexxcollins/OneDrive - City, University of London/modules/computer_vision/labs/OneDrive_1_22-02-2022/annotations/2007_000039.xml'),
 PosixPath('/Users/alexxcollins/OneDrive - City, University of London/modules/computer_vision/labs/OneDrive_1_22-02-2022/annotations/2007_000042.xml'),
 PosixPath('/Users/alexxcollins/OneDrive - City, University of London/modules/computer_vision/labs/OneDrive_1_22-02-2022/annotations/2007_000061.xml'),
 PosixPath('/Users/alexxcollins/OneDrive - City, University of London/modules/computer_vision/

In [16]:
class PascalVOC(data.Dataset):

     def __init__(self, **kwargs):
        # Classes from Pascal VOC 2012 dataset, in the correct order without the bgr
        self.voc_classes = kwargs['classes']
        self.dir = Path(kwargs['dir'])
        self.dir_label = Path(kwargs['dir_label'])
        #self.img_min_size = kwargs['img_min_size']
        self.imgs = sorted(self.dir.iterdir())
        self._classes = kwargs['classes']
        
    def extract_bboxes_pascal(self, idx, list_of_classes):
        
        fname = self.dir_label/self.imgs[idx].with_suffix('.xml').name
        classes = []
        bboxes = []
        
        pascal_voc_xml_doc = minidom.parse(fname)
        # This returns the list of objects tagged 'name'
        nameval = pascal_voc_xml_doc.getElementsByTagName('name')
        # get the number of total objects in the image
        good_class_ids = []
        
        for id, n in enumerate(nameval):
            name = n.firstChild.nodeValue
            # some classes not in Pascal VOC data set (e.g.head)
            if name in list_of_classes.keys():
                good_class_ids.append(id)                
                classes.append(list_of_classes[name])
                
        coords = ['xmin', 'ymin', 'xmax', 'ymax']
        c_dict = dict(zip(coords, [[],[],[],[],[]]))
        for c in coords:
            val = pascal_voc_xml_doc.getElementsByTagName[c]
            for id in good_class_ids:
                m = float(val[id].firstChild.nodeValue)
                c_dict[c].append(m)
        
        # put the coordinates together
        for i in range(len(good_class_ids)):
            obj = [c_cord['xmin'][i], c_cord['ymin'][i],
                   c_cord['xmax'][i], c_cord['ymax'][i]]
            bboxes.append(obj)
            
        # output a dictionary with classes and bboxes
        label={}
        label['labels'] = torch.as_tensor(classes, dtype=torch.int64)
        label['boxes'] = torch.as_tensor(bboxes)
        return label



In [17]:
pascal_object_categories = ['__bgr__', 'aeroplane', 'bicycle', 'bird', 'boat',
                     'bottle', 'bus', 'car', 'cat', 'chair',
                      'cow', 'diningtable', 'dog', 'horse',
                      'motorbike', 'person', 'pottedplant',
                     'sheep', 'sofa', 'train', 'tvmonitor']

VOC_classes = dict(zip(pascal_object_categories,
                       range(len(pascal_object_categories))))
VOC_classes_idx = dict(zip(range(len(pascal_object_categories)),
                          pascal_object_categories))
# VOC_classes

In [24]:
kwargs={'classes':pascal_object_categories[1:],
       'dir':'voc_data/test_data',
       'dir_label': Path('annotations')}

In [25]:
d = PascalVOC(**kwargs)